In [ ]:
import numpy as np
import pandas as pd
from skimage import data
import matplotlib.pyplot as plt

import skimage.data as data
import skimage.segmentation as seg
import skimage.filters as filters
import skimage.draw as draw
import skimage.color as color
import seaborn as sns

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df_train = pd.read_csv('../input/sartorius-cell-instance-segmentation/train.csv')
df_train.head()

## EDA

In [ ]:
df_train.info()

In [ ]:
len([*df_train['annotation']])

In [ ]:
print("Number of Annotation =", df_train[df_train.id=='0030fd0e6378'].shape[0])

In [ ]:
print("Number of Annotation =", df_train[df_train.id=='03b27b381a5f'].shape[0])

In [ ]:
df_train.cell_type.unique()

In [ ]:
df_train[df_train.cell_type=='shsy5y'].sample_id.unique().shape[0]

### Bounding Box

In [ ]:
def get_Bounding_Box(annotate):
    '''
    Returns [(x1,y1), (x2, y2))]
    '''
    shape = (704, 520)
    img = np.zeros((shape[0] * shape[1]), dtype=np.float32)
    s = annotate.split()
    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    temp_img = np.zeros((shape[0] * shape[1]), dtype=np.float32)
    for start, end in zip(starts, ends):
        temp_img[start : end] = 1
    p2 = np.where(temp_img==1)[0][0]//704
    p4 = np.where(temp_img==1)[0][-1]//704
    temp_img = np.transpose(temp_img.reshape(520, 704)).reshape(shape[0] * shape[1])
    p1 = np.where(temp_img==1)[0][0]//520
    p3 = np.where(temp_img==1)[0][-1]//520        
    
    return ((p1, p2),(p3,p4))

In [ ]:
annotate_list=[]
for annotate in [*df_train['annotation']]:
    annotate_list.append(get_Bounding_Box(annotate))

df_train['box'] = annotate_list
df_train.head()

In [ ]:
def image_show(image, nrows=1, ncols=1, color_map='gray', title='default image'):
    fig, ax = plt.subplots(nrows=nrows, ncols=ncols, figsize=(7, 7))
    ax.imshow(image, cmap=color_map)
    ax.axis('off')
    return fig, ax

In [ ]:
img = plt.imread('../input/sartorius-cell-instance-segmentation/train/0140b3c8f445.png')
image_show(img)
plt.show()

## Create annotation mask

In [ ]:
def create_Mask(image_id, index=None):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return 
    color: color for the mask
    Returns numpy array (mask)
    '''
    points_list=[]
    value=1
    labels = df_train[df_train["id"] == image_id]["annotation"].tolist()
    boxs = df_train[df_train["id"] == image_id]["box"].tolist()
    height = df_train[df_train["id"] == image_id]['height']
    height = int(height.unique())
    width = df_train[df_train["id"] == image_id]['width']
    width = int(width.unique())
    shape = (height, width)
    img = np.zeros((shape[0] * shape[1]), dtype=np.float32)
    for label, box in zip(labels, boxs):
        s = label.split()
        starts = list(map(lambda x: int(x) - 1, s[0::2]))
        lengths = list(map(int, s[1::2]))
        ends = [x + y for x, y in zip(starts, lengths)]
        for start, end in zip(starts, ends):
            img[start : end] = value
        points_list.append(box)
        value += 1
        
    mask = img.reshape(shape[0], shape[1])
    
    return mask, points_list

In [ ]:
image_id = '1e7ac54a9422'
mask, points = create_Mask(image_id)
image_show(mask, color_map=None)
plt.show()

In [ ]:
import cv2
image = mask.copy()
for point in points:
    image = cv2.rectangle(image, point[0], point[1], (225,0,0), 2)
    
plt.figure(figsize=(15,15))
plt.imshow(mask)
plt.imshow(image, alpha=0.5)

plt.axis('off')
plt.show()

## Filters

In [ ]:
from skimage.filters import gaussian, median, unsharp_mask
from skimage import io, img_as_float, img_as_ubyte
from skimage.morphology import disk
from skimage.restoration import denoise_bilateral, denoise_nl_means, estimate_sigma, denoise_tv_chambolle

In [ ]:
df_train.id.unique()[0]

In [ ]:
dir_path = "../input/sartorius-cell-instance-segmentation/train/"
img_id = df_train.id.unique()[30]
img = img_as_float(io.imread(dir_path+img_id+".png", as_gray=True))
image_show(img)
plt.show()

In [ ]:
def image_2_show(image, filter_img, title, color_map='gray'):
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 15))
    ax[0].imshow(image, cmap=color_map)
    ax[1].imshow(filter_img, cmap=color_map)
    ax[2].imshow(filter_img+image, cmap=color_map)
    ax[0].title.set_text('Default image')
    ax[1].title.set_text(title)
    ax[2].title.set_text(title+" and default image")
    ax[0].axis('off')
    ax[1].axis('off')
    ax[2].axis('off')
    return fig, ax

**Gaussian Blur Denoising Filter**

In [ ]:
img_gaussian_filter = gaussian(img, sigma=1, mode='constant', cval=1.0)
image_2_show(img, img_gaussian_filter, title='Gaussian Filter')
plt.show()

### Median Filter

In [ ]:
img_median_filter = median(img, disk(1), mode='constant', cval=0.0)
image_2_show(img, img_median_filter, title='Median Filter')
plt.show()

**Bilateral Filter**

In [ ]:
img_bilateral_filter = denoise_bilateral(img, sigma_color=0.05, sigma_spatial=15, multichannel=False)
image_2_show(img, img_bilateral_filter, title='Bilateral Filter')
plt.show()

**NLM Filter**

In [ ]:
sigma_est = np.mean(estimate_sigma(img, multichannel=True))
denoise_img = denoise_nl_means(img, h=1.15 * sigma_est, fast_mode=True,
                               patch_size=5, patch_distance=3, multichannel=False)
image_2_show(img, denoise_img, title='None Local Mean Filter')
plt.show()

**Total Variation Filter**

In [ ]:
TV_img = denoise_tv_chambolle(img, weight=0.1, eps=0.0002, n_iter_max=200, multichannel=False)
image_2_show(img, TV_img, title='Total Variation (TV) Filter')
plt.show()

**Unsharp Filter**

In [ ]:
img_gaussian_filter = gaussian(img, sigma=1, mode='constant', cval=1.0)
unsharp_img = unsharp_mask(img, radius=5, amount=1.0)
image_2_show(img, unsharp_img, title='Unsharp Filter')
plt.show()

In [ ]:
from skimage.filters import roberts, sobel, scharr, prewitt, farid
roberts_img = roberts(img)
sobel_img = sobel(img)
scharr_img = scharr(img)
prewitt_img = prewitt(img)
farid_img = farid(img)
image_2_show(img, roberts_img, title='Roberts Filter')
plt.show()

In [ ]:
from skimage.filters import meijering, sato, frangi, hessian
meijering_img = meijering(img)
sato_img = sato(img)
frangi_img = frangi(img)
hessian_img = hessian(img)
mask = create_Mask(img_id)
image_2_show(img, meijering_img, title='Meijering Filter')
image_2_show(sato_img, hessian_img, title='Hessian Filter')
# image_2_show(frangi_img, mask, title='Mask Filter')

plt.show();

## Create Annotation

In [ ]:
image_id = '0140b3c8f445'
mask, _= create_Mask(image_id, 1)
image_show(mask, color_map=None)
plt.show()

In [ ]:
plt.imsave('mask.png', mask)

In [ ]:
def mask_annotat(mask):
    img = mask.reshape(mask.shape[0]*mask.shape[1])
    annotation = []
    flag = 0
    for i in range(img.shape[0]):
        if flag==0 and img[i]==1:
            annotation.append(i)
            flag = 1
        elif flag==1 and img[i]==0:
            annotation.append(i - annotation[-1])
            flag=0
        else:
            pass

    return " ".join([str(x) for x in annotation])

In [ ]:
mask_annotat(mask)

## END :)